# LLM: Question-answering with RAG

The following model is designed to meet the queries of students, teachers and collaborators of the Universidad Autónoma de Occidente. It uses the Ollama platform, with the purpose of taking advantage of pre-trained open source models, such as Llama3, in a local way, together with the Langchain framework. This framework facilitates the implementation of the application of the extended language model to answer questions, in this case, using RAG (Retrieval Augmented Generation). It is important to note that this application is an initial prototype, intended to explore the use of large language models with RAG with a limited amount of data to operate locally. However, the long-term goal is to migrate it to a server with enhanced capabilities, which will allow training the model with a larger amount of data.

<center> <!-- Este HTML centra la imagen -->
    <img src="imgs/rag.png" alt="Error al cargar la imagen">
</center>

In [1]:
# Import required modules
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager 
from langchain.llms import Ollama
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

### Define variables

We define the variables of the model and embedding we are going to use, in this case the model is Llama3, an open source model released by Meta with 8 billion parameters and the embedding is a large context length text encoder called nomic-embed-text.

In [2]:
LOCAL_MODEL = "llama3" # Meta's open source model.
EMBEDDING   = "nomic-embed-text" # A high-performing open embedding model with a large token context window.

### Load the data

We define the list of object loaders representing different document sources: three local PDF files and a web URL. These sources are loaded thanks to Langchain's "PyPDFLoader" and "WebBaseLoader" loaders. It then iterates over each of these loaders and loads the corresponding documents using the load() method. Finally, all loaded documents are grouped in a list called docs.

In [ ]:
loaders = [
    PyPDFLoader("info/Código-de-Ética-para-Estudiantes.pdf"),
    PyPDFLoader("info/resolucion_No666.pdf" ),
    PyPDFLoader("info/Resolucion_de_Rectoria_No._7714_reglamento_academico_de_pregrado_Modalidad_Virtual.pdf" ),
    WebBaseLoader("https://www.uao.edu.co/solicitudes-de-supletorios-validaciones-y-habilitaciones/")
    ]

docs = []

for loader in loaders:
    docs.extend(loader.load())

# print (docs)


### Split the data

<center> <!-- Este HTML centra la imagen -->
    <img src="imgs/split.png" alt="Error al cargar la imagen">
</center>

We define the object "text_splitter" which uses the RecursiveCharacterTextSplitter method of langchain where we define a fragment size of 1500 characters and an overlap of 150 characters. Then, it uses this splitter to split a list of documents (docs) into smaller fragments that will comply with the parameters defined in the splitter, and will be stored in the variable "all_splits". Subsequently, it prints the first two generated splits and displays the total amount of splits obtained to analyze the content.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter( 
    chunk_size=1500, chunk_overlap=150)

all_splits = text_splitter.split_documents(docs)

# print(all_splits[0])
# print(all_splits[1])
# len(all_splits)

<center> <!-- Este HTML centra la imagen -->
    <img src="imgs/chunks.png" alt="Error al cargar la imagen">
</center>

### Create the vector store

<center> <!-- Este HTML centra la imagen -->
    <img src="imgs/storage.png" alt="Error al cargar la imagen">
</center>

This section of the code establishes a persistence directory, called "persist_directory", which stores the vector store used by the model to search for relevant documents that can be helpful to answer user queries. The vector store is created using Chroma, a lightweight, in-memory implementation. It is given as input the previously generated fragments (all_splits) and the embedding model defined in the variables, which is associated with Ollama. It is important to note that the persistence directory where the vector store will be stored for future use is also given as an input.

In [5]:
persist_directory = 'data' # This is where the vector store will be for future use.

vectorstore = Chroma.from_documents( # Chroma is a Vector Store.  Chroma is lightweight and in memory making it easy to start with.
    documents=all_splits, # Splits created earlier.
    embedding=OllamaEmbeddings(model=EMBEDDING), #Embedding model.
    persist_directory=persist_directory
)

##### How does a vector store work?

<center> <!-- Este HTML centra la imagen -->
    <img src="imgs/create_vectorstore.png" alt="Error al cargar la imagen">
    <img src="imgs/n_most_similar.png" alt="Error al cargar la imagen">
</center>


A vector store is a database in which we can easily search for similar vectors later on.  This is useful when trying to find relevant documents relates to a question. So, when we want to get an answer for a question, we create embeddings of the question and then the embeddings of the question are compared with all the different vectors in the vector store and choose the n most similar ones. Finally, we take the n most similar chunks, pass them along with the question to an LLM and get the answer.

### Question-answering

In [6]:
llm = Ollama(base_url="http://localhost:11434",
                                  model=LOCAL_MODEL,
                                  verbose=True, # Provide additional information about what the program is doing
                                  stop = ['<|eot_id|>'], # Prevent an infinite loop.
                                  callback_manager=CallbackManager(
                                      [StreamingStdOutCallbackHandler()]) # Monitor the model output while it is running
                                  )

retriever = vectorstore.as_retriever()

In this moment, the model for question-answering has been created, which uses Llama3 integrated through Ollama. A stop parameter has been configured to prevent the model from entering a loop once the user has made his query, ensuring that it only provides the requested answer. In addition, a callback has been implemented to monitor the output of the model while it is running and the verbose option to obtain a detailed result. The retriever has also been generated, which is built from the previously created vector store. This retriever will be used later to retrieve the relevant documents stored in the vector store.

Next, the code begins by defining a prompt template, which acts as a schema for structuring the inputs to be passed to the chatbot model. This template includes sections for the context of the conversation, the history of previous interactions, the current question and the chatbot's response. This template is then used in the defined prompt in which the input variables are also made explicit. Then, a conversation memory buffer is configured to temporarily store the chatbot's conversation history. This buffer is responsible for retaining the dialog flow, allowing the chatbot to remember previous interactions in order to generate more coherent and contextual responses. In the next code block you can see some more details. 

In [24]:
"""
The prompt template has instructions about how to use the context. It also has placeholders for variables like context, 
history, question, and the answer.
"""

template = """ You are a chatbot providing information to students, collaborators, 
and professors of Universidad Autónoma de Occidente. Your tone should be professional 
and informative. Keep responses concise, addressing only what users ask. If you don't 
know the answer, simply state so. You have to answer in the language used by the user. 
Use the following context to response the questions.
    
    Context: {context}
    History: {history}

    Question: {question}
    Chatbot:
    """

"""
A prompt is the style of creating inputs to pass into the model.
"""
prompt = PromptTemplate(
        input_variables=["history", "context", "question"],
        template=template,
    )

"""
Buffer (region of a memory used to store data temporarily) for storing conversation memory
"""
memory = ConversationBufferMemory(
        memory_key="history", # Specifies the key to be used to store the conversation memory.
        return_messages=True, # Set that the messages stored in the memory will be returned.
        input_key="question" # Specifies the key that will be used to store the conversation entries in memory.
    )

This section defines the retrieval process using RetrievalQA. This RetrievalQA uses the language model (LLM) previously established in the code, along with the information retriever configured using the vectore store. Arguments such as the prompt, the memory for the model's question and answer history, and the verbose option for detailed output are provided. At this stage, the relevant documents and the original question are processed through the LLM, prompting the model to answer the question.

<center> <!-- Este HTML centra la imagen -->
    <img src="imgs/retrieval.png" alt="Error al cargar la imagen">
</center>

In [25]:
"""
LLM chain is the combination of an LLM and a Prompt.
RetrievalQA does question answering backed by a retrieval step. This is created
by passing a language model and vector database as a retriever.
"""
qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type='stuff',
            retriever=retriever,
            verbose=True,
            chain_type_kwargs={
                "verbose": True,
                "prompt": prompt,
                "memory": memory
            }
        )


We are using the "stuff" method as a chain_type, which puts all documents in the final prompt. This involves a single call to the language model. But if we have too many documents, they may not fit in the context window. In such a case, we can use different techniques such as map-reduce, refine and map_rerank.

##### How does the RetrievalQA work?

<center> <!-- Este HTML centra la imagen -->
    <img src="imgs/RetrievalQA.png" alt="Error al cargar la imagen">
</center>   

As we can see in the picture, the user's question is sent to the vector store. Here, it's compared with the documents inside to find the most relevant ones – those that are similar to the user's question. These relevant documents, along with the user's original question, are then given to the LLM model. Using this information, the LLM generates an answer.

In [9]:
"""
This will allow us to display LLM responses in a user-friendly format.
"""

def pretty_print(answer):
    print(f"""
      UAO CHATBOT
            
      Usurario: {answer.get("query")} 
      Chat: {answer.get("result")}
            """)

### Testing

<center> <!-- Este HTML centra la imagen -->
    <img src="imgs/output.png" alt="Error al cargar la imagen">
</center>   

The tests are conducted with several questions in Spanish (as it is the main language used at the target university), one of which involves direct interaction with the user. In addition, a question is presented in English to illustrate the model's ability to respond in the language in which the user is addressed. It should be noted that a final test was conducted in Japanese, in which the model responded correctly, albeit in English, demonstrating its limitations in dealing with different languages.

In [10]:
query = "¿Cuál es la misión de la universidad autónoma de occidente?" 

In [ ]:
answer = qa_chain.invoke({"query": query})

In [12]:
pretty_print(answer)


      UAO CHATBOT
            
      Usurario: ¿Cuál es la misión de la universidad autónoma de occidente? 
      Chat: Amigo estudiante, colaborador o profesor de la Universidad Autónoma del Occidente (UAO), te brindo información acerca de la misión de esta institución educativa.

La misión de la UAO es "Ser una institución líder en el desarrollo integral del ser humano, a través de la formación académica, investigadora y comunitaria, enfocada en la calidad, innovación y sostenibilidad".

Espero que esta información te sea útil. ¿Tienes alguna otra pregunta o necesitas más detalles?
            


In [13]:
query2 = "Para la resolución de problemas ¿cual es el conducto regular?" 

In [ ]:
answer = qa_chain.invoke({"query": query2})


In [15]:
pretty_print(answer)


      UAO CHATBOT
            
      Usurario: Para la resolución de problemas ¿cual es el conducto regular? 
      Chat: Amigo estudiante, colaborador o profesor de la Universidad Autónoma del Occidente (UAO), te brindo información acerca del proceso regular para la resolución de problemas en esta institución educativa.

En la UAO, el conducto regular para la resolución de problemas es el siguiente:

1. El estudiante presenta su problema o inquietud a través de diferentes canales, como el correo electrónico [email protected], la plataforma virtual del sitio web de la universidad o personalmente en las oficinas de atención al estudiante.
2. La oficina de atención al estudiante o el área correspondiente analiza y evalúa la solicitud para determinar la mejor forma de abordar el problema.
3. Si es necesario, se puede convocar a una reunión con un representante de la universidad para discutir la situación y encontrar una solución adecuada.
4. La universidad hace todo lo posible por resolv

In [16]:
query3 = "Resume tu anterior respuesta en 10 palabras."

In [ ]:
answer = qa_chain.invoke({"query": query3})

In [18]:
pretty_print(answer)


      UAO CHATBOT
            
      Usurario: Resume tu anterior respuesta en 10 palabras. 
      Chat: La misión de la UAO es formar seres humanos integrales y sostenibles.
            


In [ ]:
"""
User interaction
"""

query = str(input("""
                  BIENVENID@ A CHAT UAO 
                  ¿En qué te puedo ayudar?
                  -> 
                  """))

answer = qa_chain.invoke({"query": query})

In [20]:
pretty_print(answer)


      UAO CHATBOT
            
      Usurario: ¿Cómo solicito un supletorio? 
      Chat: Excelente pregunta! En la Universidad Autónoma del Occidente (UAO), si necesitas solicitar un supletorio, puedes hacerlo a través de los siguientes canales:

1. Correo electrónico: Puedes enviar un correo electrónico a [email protected] con el número de identificación (NIT) o código de estudiante y describir la situación que te llevó a solicitar el supletorio.
2. Plataforma virtual del sitio web de la universidad: Puedes acceder a la plataforma virtual y buscar la sección correspondiente para solicitar un supletorio.
3. Oficinas de atención al estudiante: Puedes visitar las oficinas de atención al estudiante en persona o por teléfono para solicitar el supletorio.

Es importante tener en cuenta que es necesario presentar justificativos y documentación adicional según sea necesario, por lo que asegúrate de estar preparado antes de solicitar el supletorio.

¿Tienes alguna otra pregunta o necesitas m

In [ ]:
english_query = "What is the mission of the University Autonoma de Occidente"
english_answer = qa_chain.invoke({"query": english_query})

In [27]:
pretty_print(english_answer)


      UAO CHATBOT
            
      Usurario: What is the mission of the University Autonoma de Occidente 
      Chat: According to the provided information, Universidad Autónoma de Occidente (UAO) is a university that provides education and research opportunities. The university's personería jurídica was granted by the Gobernación del Valle del Cauca on February 20th, 1970, and its institutional accreditation of high quality was granted by the Ministerio de Educación Nacional on November 13th, 2003.

UAO is a private institution that is recognized and regulated by the Ministry of Education. Its mission is to provide high-quality education, promote research and innovation, and contribute to the development of society through the training of professionals and the generation of knowledge.

The university has several campuses, including Campus Valle del Lili, which is located in Jamundí. UAO also has a virtual campus that offers online programs and courses.

In terms of academic perform

In [ ]:
japanese_query = "オクシデンテ自治大学の使命とは？" # What is the mission of the University Autonoma de Occidente?
japanese_answer = qa_chain.invoke({"query": japanese_query})

In [29]:
pretty_print(japanese_answer)


      UAO CHATBOT
            
      Usurario: オクシデンテ自治大学の使命とは？ 
      Chat: According to the provided information, Universidad Autónoma de Occidente (UAO) is a university that provides education and research opportunities. The university's personería jurídica was granted by the Gobernación del Valle del Cauca on February 20th, 1970, and its institutional accreditation of high quality was granted by the Ministerio de Educación Nacional on November 13th, 2003.

UAO is a private institution that is recognized and regulated by the Ministry of Education. Its mission is to provide high-quality education, promote research and innovation, and contribute to the development of society through the training of professionals and the generation of knowledge.

In summary, the mission of Universidad Autónoma de Occidente (UAO) is to provide high-quality education, promote research and innovation, and contribute to the development of society through the training of professionals and the generation of

#### Images reference
Using langchain for Question Answering on Own Data - Onkar Mishra - Medium
By Onkar Mishra Container: Medium Publisher: Medium Year: 2023 URL: 
https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed